## Medical Report Classification Using BERT Model

In [ ]:

!pip install -q transformers datasets scikit-learn pandas openpyxl tqdm

from google.colab import files
uploaded = files.upload()

import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, get_scheduler
from sklearn.metrics import classification_report
from datasets import Dataset
from tqdm import tqdm

filename = list(uploaded.keys())[0]
df = pd.read_excel(filename)

if df['label'].dtype == 'object':
    label_names = df['label'].unique().tolist()
    label_map = {label: i for i, label in enumerate(label_names)}
    df['label'] = df['label'].map(label_map)
else:
    label_names = ["Radiology", "Cardiology", "Discharge Summary", "Pathology"]

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["report"], padding="max_length", truncation=True, max_length=256)

dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize_function, batched=True)
dataset = dataset.rename_column("label", "labels")
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

train_test = dataset.train_test_split(test_size=0.2)
train_dataset = train_test["train"]
test_dataset = train_test["test"]

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
test_dataloader = DataLoader(test_dataset, batch_size=16)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_names))
model.to(device)

for param in model.base_model.parameters():
    param.requires_grad = False

optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_dataloader) * 1  # 1 epoch
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

model.train()
for epoch in range(1):
    print(f"Epoch {epoch + 1}")
    loop = tqdm(train_dataloader)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

model.eval()
preds, targets = [], []

with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        preds.extend(predictions.cpu().numpy())
        targets.extend(batch["labels"].cpu().numpy())

print("\n📊 Classification Report:")
print(classification_report(targets, preds, target_names=label_names))

def predict_report(report_text):
    model.eval()
    tokens = tokenizer(report_text, return_tensors="pt", truncation=True, padding="max_length", max_length=256)
    tokens = {k: v.to(device) for k, v in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        pred_class = torch.argmax(probs, dim=-1).item()
    return label_names[pred_class], probs[0][pred_class].item()

example = "ECG shows ST elevation with signs of infarction."
label, confidence = predict_report(example)
print(f"\n🧪 Inference:\nReport: {example}\nPredicted: {label} (Confidence: {confidence:.2f})")


Saving medical_reports_1000.xlsx to medical_reports_1000 (2).xlsx


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1


Epoch 1: 100%|██████████| 50/50 [06:21<00:00,  7.63s/it, loss=1.31]



📊 Classification Report:
                   precision    recall  f1-score   support

        Radiology       0.46      0.96      0.63        54
       Cardiology       0.65      0.37      0.47        46
Discharge Summary       1.00      0.64      0.78        47
        Pathology       0.97      0.58      0.73        53

         accuracy                           0.65       200
        macro avg       0.77      0.64      0.65       200
     weighted avg       0.77      0.65      0.65       200


🧪 Inference:
Report: ECG shows ST elevation with signs of infarction.
Predicted: Radiology (Confidence: 0.27)
